In [12]:
from game import TicTacToe,Board
import random
import math
import time
infinity = math.inf

In [13]:
board = Board(width=3,height=3,to_move='X')
print(board)
# square = [{1,2},{2,3}]
# board.new({square: 'X'}, to_move='O')



. . .
. . .
. . .



# 1. Getting Started with the Game

In [3]:
# initializing a TicTacToe game
game = TicTacToe(height=3, width=3, k=3)
board = game.initial

# get the possible actions
actions = game.actions(board)
# to apply a move (the board automatically figures out which player)
board_after_move = game.result(board, list(actions)[0])

print(board)
print(actions)
print(board_after_move)

. . .
. . .
. . .

{(0, 1), (1, 2), (2, 1), (0, 0), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
. . .
X . .
. . .



In [4]:
def play_game_step(game, state, strategies: dict, verbose=False):
    start = time.perf_counter()
    player = state.to_move
    move = strategies[player](game, state)
    state = game.result(state, move)
    time_elapsed = time.perf_counter() - start
    if verbose: 
        print('Player', player, 'move:', move, f'time: {time_elapsed:.4f}s', )
        print(state)
    return state

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        state = play_game_step(game, state, strategies, verbose)
    return state

# setup a random strategy for testing
def random_player(game, state): return random.choice(list(game.actions(state)))

def search_player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1] # we expect our search algorithm to return (v, move)

In [5]:
# initialize a game
game = TicTacToe()
state = game.initial
strategies = dict(X=random_player, O=random_player)

In [6]:
# option 1: 
# play_game(game, strategies, True)

# option 2: run it step by step by executing this cell multiple times until the game end
if not game.is_terminal(state):
    state = play_game_step(game, state, strategies, True)
else:
    print(f"Game result: {game.utility(state, 'X')}")
    print(state)

Player X move: (1, 0) time: 0.0000s
. X .
. . .
. . .



# 2. MINIMAX and Alpha-Beta 

In [7]:
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None

        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v<v2:
                v =v2
                move = a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = infinity, None

        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v>v2:
                v = v2
                move = a
        return v, move

    return max_value(state)

# test against random_player
%time play_game(TicTacToe(), dict(X=search_player(minimax_search), O=random_player), True)

Player X move: (0, 1) time: 4.1600s
. . .
X . .
. . .

Player O move: (2, 1) time: 0.0000s
. . .
X . O
. . .

Player X move: (1, 2) time: 0.0715s
. . .
X . O
. X .

Player O move: (0, 2) time: 0.0000s
. . .
X . O
O X .

Player X move: (1, 1) time: 0.0022s
. . .
X X O
O X .

Player O move: (2, 0) time: 0.0000s
. . O
X X O
O X .

Player X move: (1, 0) time: 0.0001s
. X O
X X O
O X .

CPU times: user 4.13 s, sys: 18.9 ms, total: 4.15 s
Wall time: 4.23 s


. X O
X X O
O X .

In [8]:
# if you have implemented it right, it will always be a draw for h=w=k=3
%time play_game(TicTacToe(), dict(X=search_player(minimax_search), O=search_player(minimax_search)), True)

Player X move: (0, 1) time: 4.1163s
. . .
X . .
. . .

Player O move: (2, 1) time: 0.4766s
. . .
X . O
. . .

Player X move: (1, 2) time: 0.0723s
. . .
X . O
. X .

Player O move: (0, 0) time: 0.0105s
O . .
X . O
. X .

Player X move: (1, 1) time: 0.0018s
O . .
X X O
. X .

Player O move: (1, 0) time: 0.0003s
O O .
X X O
. X .

Player X move: (2, 0) time: 0.0001s
O O X
X X O
. X .

Player O move: (0, 2) time: 0.0001s
O O X
X X O
O X .

Player X move: (2, 2) time: 0.0000s
O O X
X X O
O X X

CPU times: user 4.64 s, sys: 15.8 ms, total: 4.65 s
Wall time: 4.68 s


O O X
X X O
O X X

In [9]:
def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = state.to_move
    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            # TODO: update alpha, beta pruning, decide *v* and *move*
            if(v2>v):
                v,move = v2,a
            if v2 > alpha:
                alpha = v2
            if v2>=beta:
                break
        return v, move



    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            # TODO: update alpha, beta pruning, decide *v* and *move*
            if(v2<v):
                v,move = v2,a
            if v2 < beta:
                beta = v2
            if v2<=alpha:
                break
        return v, move

    return max_value(state, -infinity, +infinity)

# test against random_player
%time play_game(TicTacToe(), dict(X=search_player(alphabeta_search), O=random_player), True)

Player X move: (0, 1) time: 0.1834s
. . .
X . .
. . .

Player O move: (1, 2) time: 0.0000s
. . .
X . .
. O .

Player X move: (1, 1) time: 0.0039s
. . .
X X .
. O .

Player O move: (1, 0) time: 0.0000s
. O .
X X .
. O .

Player X move: (2, 1) time: 0.0002s
. O .
X X X
. O .

CPU times: user 186 ms, sys: 1.92 ms, total: 188 ms
Wall time: 188 ms


. O .
X X X
. O .

In [10]:
%time play_game(TicTacToe(), dict(X=search_player(alphabeta_search), O=search_player(alphabeta_search)), True)

Player X move: (0, 1) time: 0.1883s
. . .
X . .
. . .

Player O move: (2, 1) time: 0.0308s
. . .
X . O
. . .

Player X move: (1, 2) time: 0.0096s
. . .
X . O
. X .

Player O move: (0, 0) time: 0.0028s
O . .
X . O
. X .

Player X move: (1, 1) time: 0.0012s
O . .
X X O
. X .

Player O move: (1, 0) time: 0.0001s
O O .
X X O
. X .

Player X move: (2, 0) time: 0.0001s
O O X
X X O
. X .

Player O move: (0, 2) time: 0.0000s
O O X
X X O
O X .

Player X move: (2, 2) time: 0.0000s
O O X
X X O
O X X

CPU times: user 231 ms, sys: 2.16 ms, total: 233 ms
Wall time: 233 ms


O O X
X X O
O X X